In [1]:
data_var = '2024-01-01'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8618,2024-01-01,Coreia Do Sul Kbl,04:00,Seoul Thunders,Seoul Knights,4.51,1.20,162.5,1.87,1.87,8.5,2.08,1.67,8A57hgdO,0.221729,0.833333,0.534759,0.534759,0.055063,351.662,117.369928,0.333758,0.6,1.341641,2.236068,217.58,4.724,1.937751,0.410193,-63.0,121.972,24.178423,0.198229,3.0,0.000000,0.000000,105.78,1.560,0.540417,0.346421,53.0,86,86,2.53,1.23,201.858,121.988,0.819798,0.000000,0.154621,-3.13,-0.626,-5.607029,0.420342,0.2,-0.220342,1.84,0.368,0.543478,0.628818,0.7,0.071182
8619,2024-01-01,Israel Superliga,16:05,Hapoel Jerusalem,Hapoel Beer Sheva,1.15,5.22,165.5,1.87,1.87,-12.5,1.99,1.73,nL25cD8O,0.869565,0.191571,0.534759,0.534759,0.061136,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,178.998,67.995335,0.379866,1.2,1.643168,1.369306,116.25,2.156,0.749920,0.347829,-30.0,100,75,2.30,1.55,0.000,0.000,0.903587,0.000000,0.098843,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8620,2024-01-01,Eua Ncaa,14:00,Drexel,Hampton,1.06,8.75,138.5,1.80,1.91,-14.5,1.83,1.83,bDsuYuKc,0.943396,0.114286,0.555556,0.523560,0.057682,199.792,131.532141,0.658345,1.2,1.643168,1.369306,135.88,3.106,2.446606,0.787703,-14.0,343.790,193.641629,0.563256,0.6,1.341641,2.236068,162.15,4.620,2.605907,0.564049,-27.0,86,69,1.58,2.35,0.000,389.782,1.108594,0.041931,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.15,0.030,258.333333,0.000000,0.0,0.000000
8621,2024-01-01,Eua Ncaa,16:00,Texas,Texas Arlington,1.04,10.50,146.5,1.83,1.83,-18.5,1.91,1.80,OU71BtUO,0.961538,0.095238,0.546448,0.546448,0.056777,119.688,72.003042,0.601589,2.4,1.341641,0.559017,77.04,1.634,1.173661,0.718275,60.0,288.620,171.569144,0.594446,1.2,1.643168,1.369306,217.88,4.332,2.695454,0.622219,-26.0,72,52,1.07,4.19,89.388,592.296,1.159312,0.000000,0.041931,0.50,0.100,0.400000,0.000000,0.0,0.000000,-5.00,-1.000,-9.500000,0.000000,0.0,0.000000
8622,2024-01-01,Eua Ncaa,16:00,Texas Tech,North Alabama,1.04,10.50,144.5,1.80,1.91,-17.5,1.83,1.83,pEyuzaMg,0.961538,0.095238,0.555556,0.523560,0.056777,92.350,7.533386,0.081574,3.0,0.000000,0.000000,100.80,1.110,0.111131,0.100118,104.0,320.140,274.354408,0.856983,0.6,1.341641,2.236068,541.20,4.512,3.892759,0.862757,-18.0,96,66,1.05,8.20,92.350,336.982,1.159312,0.041931,0.000000,0.55,0.110,0.363636,0.000000,0.0,0.000000,-3.58,-0.716,-13.268156,0.000000,0.0,0.000000
8623,2024-01-01,Israel Superliga,15:00,Bnei Herzliya,Nes Ziona,1.75,2.01,169.5,1.88,1.86,-2.5,2.01,1.72,0OjmMdoP,0.571429,0.497512,0.531915,0.537634,0.068941,165.906,91.213319,0.549789,1.8,1.643168,0.912871,91.20,2.044,1.283834,0.628099,5.0,255.698,291.278617,1.139151,1.8,1.643168,0.912871,140.18,2.750,3.080122,1.120044,27.0,80,86,1.14,1.63,0.000,0.000,0.097791,0.007563,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8624,2024-01-01,Israel Wbl Feminina,13:00,Maccabi Ashdod F,Maccabi Urban F,2.56,1.47,130.5,1.85,1.85,3.5,1.95,1.72,lnU7RWrL,0.390625,0.680272,0.540541,0.540541,0.070897,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,60,67,7.55,1.08,0.000,0.000,0.382504,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8625,2024-01-01,Polônia Liga D

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
8623,15:00,Israel Superliga,Bnei Herzliya,Nes Ziona,1.75,Back Home
8631,02:00,Coreia Do Sul Kbl,Suwon KT,KCC Egis,1.71,Back Home
8637,17:00,Eua Nba,New York Knicks,Minnesota Timberwolves,2.05,Back Home
8638,21:30,Eua Nba,Toronto Raptors,Cleveland Cavaliers,1.85,Back Home
8643,12:00,Arábia Saudita Premier League,Al Safa of Safwa,Al Khaleej,1.65,Back Home
8650,08:35,China Cba,Shanxi Zhongyu,Beijing Royal Fighters,1.68,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
